# How Multiprecision Leads to More Efficient and More Robust PDE Solvers

## What is a multiprecision algorithm?

- A multiprecision algorithm mixes the different floating point precisions.
- This can be used for different purposes:
  - It can lead to higher accuracy
  - It can lead to more speed
  
Higher accuracy is obvious, speed is obvious when you decrease precision. But what I want to show today is that the opposite can be true: higher precision numbers can be used to increase performance!

I will show how to use higher precision numbers to speed up calculations where you want results in `Float64`.

>There is a growing demand for and availability of multiprecision arithmetic: floating point arithmetic supporting multiple, possibly arbitrary, precisions. For an increasing body of applications, including in supernova simulations, electromagnetic scattering theory, and computational number theory, double precision arithmetic is insufficient to provide results of the required accuracy. On the other hand, for climate modelling and deep learning half precision (about four significant decimal digits) has been shown to be sufficient in some studies. We discuss a number of topics involving multiprecision arithmetic, including: The need for, availability of, and ways to exploit, higher precision arithmetic (e.g., quadruple precision arithmetic). How to derive linear algebra algorithms that will run in any precision, as opposed to be being optimized (as some key algorithms are) for double precision. For solving linear systems with the use of iterative refinement, the benefits of suitably combining three different precisions of arithmetic (say, half, single, and double). How a new form of preconditioned iterative refinement can be used to solve very ill conditioned sparse linear systems to high accuracy.

Nick Higham, "The Rise of Multiprecision Arithmetic"

http://ieeexplore.ieee.org/document/8023056/

## Why deal with floating point precision?

Floating point errors are more prevelant than you may think.

In [3]:
0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1

0.9999999999999999

In [4]:
1 - (0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1)

1.1102230246251565e-16

## Catastrophic Cancellation

- When floating point numbers are close to each other, the largest decimal places are cancelled, leaving you with less significant digits.
- 64-bit floating point numbers have about 16 significant digits to start with.

Some numerical algorithms are "numerically unstable" because they are written in a way that too many significant digits get lost.

$$ \gamma(z) = z^{-3} [-4-3z-z^2+exp(z)(4-z)] $$ 

In [13]:
γ(z) = z^(-3) * (-4 -3z - z^2 + exp(z)*(4-z))
[γ(z) for z in 10.0.^(0:-1:-13)]

14-element Array{Float64,1}:
    0.154845  
    0.16658   
    0.166666  
    0.166667  
    0.166089  
    0.0       
 -888.178     
    0.0       
    0.0       
    0.0       
    0.0       
    0.0       
    0.0       
   -8.88178e23

In [14]:
[γ(z) for z in big(10.0).^(0:-1:-13)]

14-element Array{BigFloat,1}:
 1.548454853771357060808624140579874932717412810998787249009028831722298910606636e-01
 1.66580495025736765660523311962006075734059476323003292166828819775070315847974e-01 
 1.666658305495932401730424115348913751840819271355059720847049530210180144521954e-01
 1.666666583305549602182401879407417261222251394538301831669786332575823153978047e-01
 1.666666665833305554960307539544751432963063004381003501477127675851005003988853e-01
 1.666666666658333305555496031646825259038635361376663292982592885815571781557752e-01
 1.666666666666583333305555549603173611110973324498456788369994396928826516439519e-01
 1.666666666666665833333305555554960317450396825259038799571225792615706816533831e-01
 1.666666666666666658333333305555555496031745932539682004465582307851582870418442e-01
 1.66666666666666666658333333330555555554960317460247755321726926688492858950762e-01 
 1.66666666666666666666583333333330555555555496047730271150873838582095341205687e-01 
 1.666666666666666666666

## Numerical analysis is an entire science built around fixing these problems

- How do we know when a problem is going to happen?
- How do we efficient avoid numerical problems?

## Basic Multiprecision Algorithm: Neural Nets and Optimization

Optimize:

$$ f(x), x \in D$$

- You don't accumulate error into `x`, so you might as well make it a `Float32` or a `Float16`.
- If necessary, upconvert to `Float64` inside of `f` to avoid catastrophic cancellation
- Autodifferentiation computes gradients at machine accuracy
- The standard training methods like Gradient Descent and ADAM are safe (when used with automatic differentiation), so this algorihtm is "low precision safe".

This is why "gamer GPUs" can be used!

## Avoiding Issues in Rootfinding Algorithms

Solving $g(x)=0$

Native way: iterately find the values of `x`.

Issue: sensitivity of the function `g` to input values.

In [22]:
g(x) = 1e24*x - 1e24

x = 1.0
println(g(x))
x_next = nextfloat(x)
println(g(x_next))
x_next2 = nextfloat(x_next)
println(g(x_next2))

0.0
2.68435456e8
4.02653184e8


## Application: Stiff ODE Solvers

DiffEq's way:

<img src="https://user-images.githubusercontent.com/1814174/35989884-411f3fee-0cb7-11e8-8d93-83ea9a4c3c5d.PNG">

Sundials ARKODE's way:

<img src="https://user-images.githubusercontent.com/1814174/35989888-4230242a-0cb7-11e8-8f82-cff416444179.PNG">

DiffEqBenchmarks.jl show how you can be orders of magnitude faster and more robust by doing this correctly.

Note: Sundials CVODE and IDA do the DiffEq way!

## Now that we are familiar with the subject, let's look at a specific example

Fourth-Order Time-Stepping for Stiff PDEs

Aly-Khan Kassam and Lloyd N. Trefethen

Solve a discretization of a semilinear PDE:

$$ u_t = Lu + N(u,t) $$

via ETDRK4. Famous paper shows how to solve this fast!

<img src="https://user-images.githubusercontent.com/1814174/35991438-12ffb882-0cbc-11e8-9ee4-18ed61f12e70.PNG">

Idea: define a few constants:

<img src="https://user-images.githubusercontent.com/1814174/35991506-3d6d8716-0cbc-11e8-824f-29becfbca983.PNG">

and now your loop just has a few matrix multiplications.

One little problem: those equations are not numerically stable (γ is the equation from before!)

Solution: solve it using contour integration. Pick a contour the encloses the eigenvalues of $L$ and do:

<img src="https://user-images.githubusercontent.com/1814174/35991440-13220798-0cbc-11e8-81aa-0460601e5285.PNG">

<img src="https://user-images.githubusercontent.com/1814174/35991442-133c6f8e-0cbc-11e8-9199-b90d402baee3.PNG">

In [24]:
γ(z) = z^(-3) * (-4 -3z - z^2 + exp(z)*(4-z))
[γ(z) for z in big(10.0).^(0:-1:-13)]

14-element Array{BigFloat,1}:
 1.548454853771357060808624140579874932717412810998787249009028831722298910606636e-01
 1.66580495025736765660523311962006075734059476323003292166828819775070315847974e-01 
 1.666658305495932401730424115348913751840819271355059720847049530210180144521954e-01
 1.666666583305549602182401879407417261222251394538301831669786332575823153978047e-01
 1.666666665833305554960307539544751432963063004381003501477127675851005003988853e-01
 1.666666666658333305555496031646825259038635361376663292982592885815571781557752e-01
 1.666666666666583333305555549603173611110973324498456788369994396928826516439519e-01
 1.666666666666665833333305555554960317450396825259038799571225792615706816533831e-01
 1.666666666666666658333333305555555496031745932539682004465582307851582870418442e-01
 1.66666666666666666658333333330555555554960317460247755321726926688492858950762e-01 
 1.66666666666666666666583333333330555555555496047730271150873838582095341205687e-01 
 1.666666666666666666666

<img src="https://user-images.githubusercontent.com/1814174/35991683-cc219d94-0cbc-11e8-9538-6e8ca2df1609.PNG">

## But does it actually work?

In [25]:
function cheb(N)
    N==0 && return (0,1)
    x = cos.(pi*(0:N)/N)
    c = [2; ones(N-1,1); 2].*(-1).^(0:N)
    X = repmat(x,1,N+1)
    dX = X-X'
    D  = (c*(1./c)')./(dX+(eye(N+1)))      # off-diagonal entries
    D  = D - Diagonal(sum(D,2)[:])                 # diagonal entries
    D,x
end

N = 20
D,x = cheb(N)
x = x[2:N]
w = .53*x + .47*sin.(-1.5*pi*x) - x # use w = u-x to make BCs homogeneous
u = [1;w+x;-1]
h=1/4

0.25

In [59]:
function contour_integral(h,M=32,contour_length=30)
    r = 30*exp.(1im*pi*((1:M)-.5)/M) # points along complex circle
    L = D^2; L = .01*L[2:N,2:N] # 2nd-order differentiation
    A = h*L
    E = expm(A); E2 = expm(A/2);
    I = eye(N-1); Z = zeros(N-1,N-1)
    f1 = Z; f2 = Z; f3 = Z; Q = Z

    for j = 1:M
        z = r[j];
        zIA = inv(z*I-A);
        Q = Q + h*zIA*(exp(z/2)-1);
        f1 = f1 + h*zIA*(-4-z+exp(z)*(4-3*z+z^2))/z^2;
        f2 = f2 + h*zIA*(2+z+exp(z)*(z-2))/z^2;
        f3 = f3 + h*zIA*(-4-3*z-z^2+exp(z)*(4-z))/z^2;
    end
    E,E2,real(f1/M),real(f2/M),real(f3/M),real(Q/M)
end

@time E,E2,f1,f2,f3,Q = contour_integral(h,32,15); # same as the paper

  0.362286 seconds (24.48 k allocations: 5.292 MiB)


## Julia Approach

- Julia compiles codes specifically for the input types
- It will generate fast code, even for high precision numbers
- Let's write a multiprecision algorithm that upscales some parts to BigFloats, calculates, then downscales back to Float64

In [44]:
function multiprecision(_h)
    h = big(_h)
    L = big.(D^2); L = .01*L[2:N,2:N] # 2nd-order differentiation
    A = h*L
    E = big.(expm(Float64.(A))); E2 = big.(expm(Float64.(A/2)));
    A = big.(A); L = big.(L)
    coeff = h^(-2) * L^(-3)
    A2 = A^2
    Q = Float64.((E2-I)/L)
    a = Float64.(coeff * (-4I - A + E*(4I - 3A  + A2)))
    b = Float64.(coeff * (2I + A + E*(-2I + A)))
    c = Float64.(coeff * (-4I - 3A - A2 + E*(4I-A)))
    Float64.(E),Float64.(E2),a,b,c,Q
end

@time Ẽ,Ẽ2,ã,b̃,c̃,Q̃2 = multiprecision(h);

  0.347007 seconds (367.85 k allocations: 18.631 MiB, 2.82% gc time)


In [45]:
# Their algorithm against ours

println("E diff: ", Float64.(norm(E-Ẽ, Inf)))
println("E2 diff: ",Float64.(norm(E2-Ẽ2,Inf)))
println("f1 diff: ",Float64.(norm(f1-ã, Inf)))
println("f2 diff: ",Float64.(norm(f2-b̃, Inf)))
println("f3 diff: ",Float64.(norm(f3-c̃, Inf)))
println("Q diff: ", Float64.(norm(Q-Q̃2, Inf)))

E diff: 0.0
E2 diff: 0.0
f1 diff: 1016.1988572940766
f2 diff: 15.630680277416415
f3 diff: 15.149833068765068
Q diff: 9.257143425563889e-13


## We differ by 1000 in norm on one of the operators? (This is confirmed by checking directly against the MATLAB code).

Who's wrong?

In [46]:
# Their code, but upconverted to BigFloats

function big_contour_integral(_h,M=32)
    h = big(_h)
    r = 50*exp.(1im*big.(pi)*((1:M)-.5)/M) # points along complex circle
    L = big.(D^2); L = .01*L[2:N,2:N] # 2nd-order differentiation
    A = h*L
    E = big.(expm(Float64.(A))); E2 = big.(expm(Float64.(A/2)));
    I = big.(eye(N-1)); Z = zeros(BigFloat,N-1,N-1)
    f1 = Z; f2 = Z; f3 = Z; Q = Z

    for j = 1:M
        z = r[j];
        zIA = inv(z*I-A);
        Q = Q + h*zIA*(exp(z/2)-1);
        f1 = f1 + h*zIA*(-4-z+exp(z)*(4-3*z+z^2))/z^2;
        f2 = f2 + h*zIA*(2+z+exp(z)*(z-2))/z^2;
        f3 = f3 + h*zIA*(-4-3*z-z^2+exp(z)*(4-z))/z^2;
    end
    E,E2,real(f1/M),real(f2/M),real(f3/M),real(Q/M)
end

@time E,E2,f1,f2,f3,Q = big_contour_integral(h,128);

  3.617924 seconds (31.98 M allocations: 1.457 GiB, 20.95% gc time)


In [47]:
println("E diff: ", Float64.(norm(E-Ẽ, Inf)))
println("E2 diff: ",Float64.(norm(E2-Ẽ2,Inf)))
println("f1 diff: ",Float64.(norm(f1-ã, Inf)))
println("f2 diff: ",Float64.(norm(f2-b̃, Inf)))
println("f3 diff: ",Float64.(norm(f3-c̃, Inf)))
println("Q diff: ", Float64.(norm(Q-Q̃2, Inf)))

E diff: 0.0
E2 diff: 0.0
f1 diff: 3.743791623349559e-9
f2 diff: 1.3116559357892626e-9
f3 diff: 2.199026197507343e-9
Q diff: 1.699216236371553e-14


## After we change their algorithm (essentially copy/paste from the paper) to use BigFloat, we match!

How sensitive was their original algorithm?

In [53]:
# Up the integration points to 128. They argue you can use a small number because it converges exponentially,
# this is proven false by the actual calculation

# Radius 15
@time E,E2,f1,f2,f3,Q = contour_integral(h,32,15); 
println("f1 diff: ",Float64.(norm(f1-ã, Inf)))

@time E,E2,f1,f2,f3,Q = contour_integral(h,128,15); 
println("f1 diff: ",Float64.(norm(f1-ã, Inf)))

  0.300812 seconds (1.41 k allocations: 4.362 MiB, 64.55% gc time)
f1 diff: 1016.1988572940766
  0.498916 seconds (5.25 k allocations: 16.598 MiB)
f1 diff: 7.312173475863082e-7


In [65]:
# Timings
using BenchmarkTools
@btime E,E2,f1,f2,f3,Q = contour_integral(h,128,15);
@btime Ẽ,Ẽ2,ã,b̃,c̃,Q̃2 = multiprecision(h);

  294.086 ms (5236 allocations: 16.60 MiB)
  143.624 ms (360727 allocations: 18.25 MiB)


## Result

- Their algorithm takes 128 matrix inversions
  - This exact number is problem-dependent and hard to find!
- Ours does 1
- Ours can easily be further optimized to be only 4 allocations total
- Even without optimizations, we're already faster on a small problem
  - Their algorithm scales worse due to the inversions!
  
Thus this multiprecision algorithm, taking a tour through BigFloats, is both faster and more robust!

We could also use things like DoubleFloats.jl to speed this up even more.

## Note

This will not make all problems faster. It can be dependent on the size of the matrix, the optimality of the higher precision number implementation, the condition number of the matrix, etc. Some cases the contour integral approach will be better, while in others the BigFloat method will be. But for sure the BigFloat method can be easier to get right the first time!

## Conclusion

- Higher and lower precision numbers can allow you to optimize codes even when all you care about is 64-bit floats
- Multiprecision algorithms can be much simpler, more robust to numerical issues, and faster
- Julia's generic programming tools are well-designed for fast multiprecision computing

How much of numerical analysis has been sidetracked because we didn't have Julia? This is an entire subject to research which is not suited for R/MATLAB/Python, and is only possible in C++ if you're a programming genius. However, it's widely accessible in Julia.